In [1]:
!pip install mysqlclient

In [24]:
!pip install alphabet-detector

  Running setup.py bdist_wheel for alphabet-detector ... done
  Stored in directory: /Users/vinay/Library/Caches/pip/wheels/75/a7/c3/faf515469622e565475fc96983418ba6e03f6a0f73755cd892
Successfully built alphabet-detector


In [1]:
import pandas as pd
import re
import numpy as np
from datetime import datetime
import time

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from alphabet_detector import AlphabetDetector

from config import user_id,password
# # print(user_id + " " + password)

In [2]:
# Read csv file : NYT
nyt_file = "./resources/nyt.csv"
nyt_df = pd.read_csv(nyt_file)
nyt_df.head(3)

,_id/$oid,bestsellers_date/$date/$numberLong,published_date/$date/$numberLong,amazon_product_url,author,description,price/$numberInt,publisher,title,rank/$numberInt,rank_last_week/$numberInt,weeks_on_list/$numberInt,price/$numberDouble
0,5b4aa4ead3089013507db18b,1211587200000,1212883200000,http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",27.0,Bantam,ODD HOURS,1,0,1,NaN
1,5b4aa4ead3089013507db18c,1211587200000,1212883200000,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,NaN,"Little, Brown",THE HOST,2,1,3,25.99
2,5b4aa4ead3089013507db18d,1211587200000,1212883200000,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,NaN,St. Martin's,LOVE THE ONE YOU'RE WITH,3,2,2,24.95


In [3]:
# Read csv file : good reads
csv_file = "./resources/good_reads_final.csv"
good_reads_df = pd.read_csv(csv_file)
good_reads_df[good_reads_df.publish_date.isnull()]

,author_average_rating,author_gender,author_genres,author_id,author_name,author_page_url,author_rating_count,author_review_count,birthplace,book_average_rating,book_fullurl,book_id,book_title,genre_1,genre_2,num_ratings,num_reviews,pages,publish_date,score
180,4.18,female,"young-adult,",4951964,Shelly Crane\n,/author/show/4951964.Shelly_Crane,161708,8999,United States\n,4.12,https://www.goodreads.com/book/show/35534230-s...,35534230,\n Significance\n,Fantasy,Young Adult,39913,2772,387,NaN,17211
399,4.04,male,"literature-fiction,",3849415,Deborah Harkness\n,/author/show/3849415.Deborah_Harkness,537297,52864,\n United States\n,3.99,https://www.goodreads.com/book/show/8667848-a-...,8667848,\n A Discovery of Witches\n,Fantasy,Fiction,290944,29048,579,NaN,14789
971,3.80,female,"young-adult,",5447785,Sarah M. Ross\n,/author/show/5447785.Sarah_M_Ross,3295,518,\n United States\n,3.84,https://www.goodreads.com/book/show/15891600-e...,15891600,\n Echo of an Earth Angel\n,Paranormal,Young Adult,147,41,295,NaN,1505
1021,4.28,male,"literature-fiction,science-fiction-fantasy,",1370283,Brent Weeks\n,/author/show/1370283.Brent_Weeks,526244,18087,\n United States\n,4.43,https://www.goodreads.com/book/show/6122735-th...,6122735,\n The Night Angel Trilogy\n,Fantasy,Fiction,21414,543,1392,NaN,378
1037,4.07,male,"science-fiction,fantasy,poetry,",3619,Roger Zelazny\n,/author/show/3619.Roger_Zelazny,618504,20732,United States\n,4.30,https://www.goodreads.com/book/show/5367.The_G...,5367,\n The Great Book of Amber\n,Fantasy,Fiction,22919,872,1258,NaN,1043
1131,3.97,female,"fiction,romance,",13705141,Phoenyx Slaughter\n,/author/show/13705141.Phoenyx_Slaughter,6239,636,\n United States\n,4.04,https://www.goodreads.com/book/show/26028873-d...,26028873,\n Disconnect\n,Romance,Adult Fiction,1324,113,150,NaN,4254
1309,4.10,male,"science-fiction-fantasy,",42110,Lynn Flewelling\n,/author/show/42110.Lynn_Flewelling,84705,4069,United States\n,4.08,https://www.goodreads.com/book/show/74270.Luck...,74270,\n Luck in the Shadows\n,Fantasy,Lgbt,16680,977,479,NaN,2693
1435,3.70,male,"satire,",2751,Bret Easton Ellis\n,/author/show/2751.Bret_Easton_Ellis,360544,16208,United States\n,3.82,https://www.goodreads.com/book/show/28676.Amer...,28676,\n American Psycho\n,Fiction,Horror,200294,8615,399,NaN,15115
1588,4.14,male,"humor,entertainment,",1280953,Ben Peller\n,/author/show/1280953.Ben_Peller,685,391,\n,3.96,https://www.goodreads.com/book/show/11393301-t...,11393301,\n To Live and Drink in L.A.\n,Autobiography,Young Adult,437,219,352,NaN,6905
2135,4.01,female,"young-adult,",6950900,Shay Lynam\n,/author/show/6950900.Shay_Lynam,378,111,\n United States\n,4.10,https://www.goodreads.com/book/show/17407260-t...,17407260,\n The Tree House\n,Romance,Mystery,90,36,260,NaN,5222


In [4]:
# Data cleaning (NYT) : convert from unix timestamp and extract year only - published dates column
pub_dates =nyt_df["published_date/$date/$numberLong"]
new_pub_dates =[]

for date in pub_dates:
    date=str(date)
    date = date[0:10]
    new_date = datetime.fromtimestamp(int(date))
    
    new_pub_dates.append(new_date.year)


nyt_df["New Pub Year"] = new_pub_dates

In [5]:
# Data cleaning (NYT) : convert from unix timestamp and extract year only - bestsellers dates column
bestsellers_dates =nyt_df["bestsellers_date/$date/$numberLong"]
new_bestsellers_dates =[]

for date_best in bestsellers_dates:
    date_best =str(date_best)
    date_best = date_best[0:10]
    new_date_best = datetime.fromtimestamp(int(date_best))
    
    new_bestsellers_dates.append(new_date_best.year)

nyt_df["New Bestsellers Year"] = new_bestsellers_dates
# nyt_df.head()

In [6]:
# Create a filtered dataframe from specific columns (NYT)
nyt_cols = ["title", "author", "New Pub Year", "New Bestsellers Year", "rank/$numberInt"]
nyt_transformed= nyt_df[nyt_cols].copy()
nyt_transformed.sample(20)

# Rename the column headers
nyt_transformed = nyt_transformed.rename(columns={"title" :"Title",                                                                                                        "author": "Author",
                                                    "New Pub Year" :"Published year",
                                                    "New Bestsellers Year": "Bestsellers year",
                                                    "rank/$numberInt": "Rank"
                                                 })

# Clean the data by dropping duplicates and setting the index
nyt_transformed.drop_duplicates("Title", inplace=True)
# nyt_transformed.set_index("Title", inplace=True)

nyt_transformed.sample(10)


,Title,Author,Published year,Bestsellers year,Rank
6247,THE SKIN COLLECTOR,Jeffery Deaver,2014,2014,8
3807,COPPER BEACH,Jayne Ann Krentz,2012,2012,8
248,OFF SEASON,Anne Rivers Siddons,2008,2008,9
3074,THE LOST FLEET. BEYOND THE FRONTIER: DREADNAUGHT,Jack Campbell,2011,2011,15
7601,PURITY,Jonathan Franzen,2015,2015,2
2886,THE UNION QUILTERS,Jennifer Chiaverini,2011,2011,7
8680,COMMONWEALTH,Ann Patchett,2016,2016,1
3580,THE LITIGATORS,John Grisham,2011,2011,1
9371,THE SILENT CORNER,Dean Koontz,2017,2017,2
7274,DAY SHIFT,Charlaine Harris,2015,2015,15


In [7]:
# DATA CLEANING (good reads)-- properly format publication dates and drop rows with unconventional Null values
dates = list(good_reads_df["publish_date"])
pub_date = []    
for date in dates:
    date = str(date)
    # returns only a four digit date from date string
    x = re.search("[1-2][0-9][0-9][0-9]", date)
    if x:
        pub_date.append(x.group())
    else:
        #replaces non-date values with null so it can later be dropped
        x = np.nan
        pub_date.append(x)

good_reads_df["publish_date"] = pub_date
good_reads_df.dropna(subset=['publish_date'], how = 'any')

,author_average_rating,author_gender,author_genres,author_id,author_name,author_page_url,author_rating_count,author_review_count,birthplace,book_average_rating,book_fullurl,book_id,book_title,genre_1,genre_2,num_ratings,num_reviews,pages,publish_date,score
0,4.01,female,"historical-fiction,",74489,Victoria Thompson\n,/author/show/74489.Victoria_Thompson,74399,6268,United States\n,4.02,https://www.goodreads.com/book/show/686717.Mur...,686717,\n Murder on St. Mark's Place\n,Mystery,Historical,5260,375,277,2000,3230
1,4.15,male,"literature-fiction,mystery-thrillers,",706255,Stieg Larsson\n,/author/show/706255.Stieg_Larsson,3726435,142704,Sweden\n,4.13,https://www.goodreads.com/book/show/2429135.Th...,2429135,\n The Girl with the Dragon Tattoo\n,Fiction,Mystery,2229163,65227,465,2005,3062
2,4.00,female,"romance,",5618190,Mimi Jean Pamfiloff\n,/author/show/5618190.Mimi_Jean_Pamfiloff,76496,7975,United States\n,3.99,https://www.goodreads.com/book/show/27833684-t...,27833684,\n Tailored for Trouble\n,Romance,Contemporary,2151,391,354,2016,4585
3,3.88,male,"fiction,memoir,",37871,José Donoso\n,/author/show/37871.Jos_Donoso,5522,489,Chile\n,4.14,https://www.goodreads.com/book/show/382975.The...,382975,\n The Obscene Bird of Night\n,Fiction,Magical Realism,1844,173,438,1970,1533
4,4.10,female,"young-adult,fantasy,",36122,Patricia C. Wrede\n,/author/show/36122.Patricia_C_Wrede,291013,13453,United States\n,4.01,https://www.goodreads.com/book/show/64207.Sorc...,64207,\n Sorcery & Cecelia: or The Enchanted Ch...,Fantasy,Young Adult,17051,1890,326,1988,2105
5,3.77,male,"horror,",58947,Steve Niles\n,/author/show/58947.Steve_Niles,47938,3240,United States\n,3.80,https://www.goodreads.com/book/show/831829.30_...,831829,"\n 30 Days of Night, Vol. 1\n",Sequential Art,Sequential Art,17122,561,104,2004,4372
6,4.16,female,"romance,",4833990,Jillian Dodd\n,/author/show/4833990.Jillian_Dodd,110522,9451,\n,3.95,https://www.goodreads.com/book/show/34804503-s...,34804503,\n Stalk Me\n,Young Adult,Romance,11684,1107,327,2012,2396
7,3.94,female,"nonfiction,",7956,Mary Roach\n,/author/show/7956.Mary_Roach,321197,29747,United States\n,3.84,https://www.goodreads.com/book/show/5981308-bonk,5981308,\n Bonk: The Curious Coupling of Science ...,Nonfiction,Science,45963,4268,319,2008,2054
8,3.78,female,"fantasy,",155651,Nancy Baker\n,/author/show/155651.Nancy_Baker,1019,104,\n Canada\n,3.77,https://www.goodreads.com/book/show/266600.Kis...,266600,\n Kiss of the Vampire\n,Paranormal,Horror,594,42,278,1993,1311
9,4.08,female,"young-adult,",274533,Simone Elkeles\n,/author/show/274533.Simone_Elkeles,481114,25166,United States\n,4.01,https://www.goodreads.com/book/show/544424.Lea...,544424,\n Leaving Paradise\n,Young Adult,Romance,40093,2375,303,2007,1994


In [8]:
# data cleaning first steps (good reads) -- drop rows with null values
#all nulls are in the publish_date column
good_reads_df.dropna(subset=['publish_date'], how = 'any')

,author_average_rating,author_gender,author_genres,author_id,author_name,author_page_url,author_rating_count,author_review_count,birthplace,book_average_rating,book_fullurl,book_id,book_title,genre_1,genre_2,num_ratings,num_reviews,pages,publish_date,score
0,4.01,female,"historical-fiction,",74489,Victoria Thompson\n,/author/show/74489.Victoria_Thompson,74399,6268,United States\n,4.02,https://www.goodreads.com/book/show/686717.Mur...,686717,\n Murder on St. Mark's Place\n,Mystery,Historical,5260,375,277,2000,3230
1,4.15,male,"literature-fiction,mystery-thrillers,",706255,Stieg Larsson\n,/author/show/706255.Stieg_Larsson,3726435,142704,Sweden\n,4.13,https://www.goodreads.com/book/show/2429135.Th...,2429135,\n The Girl with the Dragon Tattoo\n,Fiction,Mystery,2229163,65227,465,2005,3062
2,4.00,female,"romance,",5618190,Mimi Jean Pamfiloff\n,/author/show/5618190.Mimi_Jean_Pamfiloff,76496,7975,United States\n,3.99,https://www.goodreads.com/book/show/27833684-t...,27833684,\n Tailored for Trouble\n,Romance,Contemporary,2151,391,354,2016,4585
3,3.88,male,"fiction,memoir,",37871,José Donoso\n,/author/show/37871.Jos_Donoso,5522,489,Chile\n,4.14,https://www.goodreads.com/book/show/382975.The...,382975,\n The Obscene Bird of Night\n,Fiction,Magical Realism,1844,173,438,1970,1533
4,4.10,female,"young-adult,fantasy,",36122,Patricia C. Wrede\n,/author/show/36122.Patricia_C_Wrede,291013,13453,United States\n,4.01,https://www.goodreads.com/book/show/64207.Sorc...,64207,\n Sorcery & Cecelia: or The Enchanted Ch...,Fantasy,Young Adult,17051,1890,326,1988,2105
5,3.77,male,"horror,",58947,Steve Niles\n,/author/show/58947.Steve_Niles,47938,3240,United States\n,3.80,https://www.goodreads.com/book/show/831829.30_...,831829,"\n 30 Days of Night, Vol. 1\n",Sequential Art,Sequential Art,17122,561,104,2004,4372
6,4.16,female,"romance,",4833990,Jillian Dodd\n,/author/show/4833990.Jillian_Dodd,110522,9451,\n,3.95,https://www.goodreads.com/book/show/34804503-s...,34804503,\n Stalk Me\n,Young Adult,Romance,11684,1107,327,2012,2396
7,3.94,female,"nonfiction,",7956,Mary Roach\n,/author/show/7956.Mary_Roach,321197,29747,United States\n,3.84,https://www.goodreads.com/book/show/5981308-bonk,5981308,\n Bonk: The Curious Coupling of Science ...,Nonfiction,Science,45963,4268,319,2008,2054
8,3.78,female,"fantasy,",155651,Nancy Baker\n,/author/show/155651.Nancy_Baker,1019,104,\n Canada\n,3.77,https://www.goodreads.com/book/show/266600.Kis...,266600,\n Kiss of the Vampire\n,Paranormal,Horror,594,42,278,1993,1311
9,4.08,female,"young-adult,",274533,Simone Elkeles\n,/author/show/274533.Simone_Elkeles,481114,25166,United States\n,4.01,https://www.goodreads.com/book/show/544424.Lea...,544424,\n Leaving Paradise\n,Young Adult,Romance,40093,2375,303,2007,1994


In [9]:
#DATA CLEANING (good reads): remove '/na' from cells 
cleaned_goodreads_df = good_reads_df.replace(r'\n|\"','', regex=True) 
cleaned_goodreads_df.head(5)

,author_average_rating,author_gender,author_genres,author_id,author_name,author_page_url,author_rating_count,author_review_count,birthplace,book_average_rating,book_fullurl,book_id,book_title,genre_1,genre_2,num_ratings,num_reviews,pages,publish_date,score
0,4.01,female,"historical-fiction,",74489,Victoria Thompson,/author/show/74489.Victoria_Thompson,74399,6268,United States,4.02,https://www.goodreads.com/book/show/686717.Mur...,686717,Murder on St. Mark's Place,Mystery,Historical,5260,375,277,2000,3230
1,4.15,male,"literature-fiction,mystery-thrillers,",706255,Stieg Larsson,/author/show/706255.Stieg_Larsson,3726435,142704,Sweden,4.13,https://www.goodreads.com/book/show/2429135.Th...,2429135,The Girl with the Dragon Tattoo,Fiction,Mystery,2229163,65227,465,2005,3062
2,4.00,female,"romance,",5618190,Mimi Jean Pamfiloff,/author/show/5618190.Mimi_Jean_Pamfiloff,76496,7975,United States,3.99,https://www.goodreads.com/book/show/27833684-t...,27833684,Tailored for Trouble,Romance,Contemporary,2151,391,354,2016,4585
3,3.88,male,"fiction,memoir,",37871,José Donoso,/author/show/37871.Jos_Donoso,5522,489,Chile,4.14,https://www.goodreads.com/book/show/382975.The...,382975,The Obscene Bird of Night,Fiction,Magical Realism,1844,173,438,1970,1533
4,4.10,female,"young-adult,fantasy,",36122,Patricia C. Wrede,/author/show/36122.Patricia_C_Wrede,291013,13453,United States,4.01,https://www.goodreads.com/book/show/64207.Sorc...,64207,Sorcery & Cecelia: or The Enchanted Choc...,Fantasy,Young Adult,17051,1890,326,1988,2105


In [10]:
#create authors_df which will become authors table in SQL
authors_df = cleaned_goodreads_df[["author_name","author_gender",
                                   'author_average_rating', 'author_rating_count', 'birthplace']].copy()
authors_df["author_name"] = authors_df["author_name"].str.strip()
authors_df.drop_duplicates(subset ="author_name", 
                     keep = "first", inplace = True)
authors_df.sort_values(by ='author_name')

,author_name,author_gender,author_average_rating,author_rating_count,birthplace
17158,19,male,3.92,424,United States
15573,A. Kirk,male,4.20,16254,
22504,A. Digger Stolz,male,4.18,637,CT
16208,A. Lee Martinez,male,3.85,60542,United States
1099,A. Lynden Rolland,female,3.94,280,
9188,A. Manette Ansay,female,3.39,31196,United States
1595,A. Meredith Walters,female,4.00,96024,United States
4025,A. Payne,female,4.18,513,United States
18923,A. Wilding Wells,female,4.24,1186,United States
4481,A. Zavarelli,female,4.19,61962,United States


In [11]:
## cleaning authors name
ad = AlphabetDetector()
authors= list(cleaned_goodreads_df["author_name"])
Latin_char_authors = []

for person in authors:
   if ad.only_alphabet_chars(person, "LATIN"):
       Latin_char_authors.append(person)
       print(person)
   else:
       Latin_char_authors.append(np.nan)
       print(person)

cleaned_goodreads_df["author_name"] = Latin_char_authors
titles= list(cleaned_goodreads_df["book_title"])
Latin_char_titles = []

for title in titles:
   if ad.only_alphabet_chars(title, "LATIN"):
       Latin_char_titles.append(title)
   else:
       Latin_char_titles.append(np.nan)

cleaned_goodreads_df["book_title"] = Latin_char_titles
cleaned_goodreads_df = cleaned_goodreads_df.dropna(subset=["book_title"], how = 'any')
cleaned_goodreads_df =  cleaned_goodreads_df.dropna(subset=["author_name"], how = 'any')

Victoria Thompson
Stieg Larsson
Mimi Jean Pamfiloff
José Donoso
Patricia C. Wrede
Steve Niles
Jillian Dodd
Mary Roach
Nancy Baker
Simone Elkeles
Brandon Sanderson
K.L. Armstrong
Lisa Kleypas
Suzanne Collins
Aldous Huxley
Sandrine Gasq-Dion
Jamie Fessenden
Zarah Detand
Kendall McKenna
Rhys Ford
Megan Derr
Sam B. Morgan
Adrienne Wilder
Kate Sherwood
Piper Vaughn
Nick Pageant
K.A. Mitchell
Sarina Bowen
Cat Grant
L.A. Witt
Amy Rae Durreson
K.C. Wells
Indra Vaughn
Rob Colton
K.J. Charles
Leta Blake
Perie Wolford
Heidi Cullinan
Veronica Sloane
Amy Lane
Josh Lanyon
Ais
Tara Lain
Jordan L. Hawk
Nicola Haken
Alexa Land
Charlie Cochet
Mary Calmes
Diane  Adams
Jaime Reese
Keira Andrews
Jay Northcote
Lane Hayes
Renae Kaye
Garrett Leigh
John  Wiltshire
N.R. Walker
S.E. Harmon
A.E. Via
Jay Bell
Jordan Castillo Price
Nicole Edwards
A.J.  Rose
Mercy Celeste
J.L. Merrow
Eli Easton
Anyta Sunday
Amelia C. Gormley
Lisa Henry
Kade Boehme
Joanna Chambers
Nick Wilgus
S.E. Jakes
Kindle Alexander
T.J. Klune
El

Jay Anson
A.M. Homes
Sarah Palin
Ann Rule
Jerzy Kosiński
Katherine Dunn
Charlotte Perkins Gilman
Flora Rheta Schreiber
Marquis de Sade
Jack Ketchum
Iain Banks
Chevy Stevens
Juan Gomez-Jurado
Jennifer  McMahon
Rachel Simon
David Nicholls
Paula McLain
Elizabeth Chandler
Elana Johnson
Carrie Harris
Mandy Hubbard
Cat Clarke
Jenna Black
Leah Clifford
Alexandra Monir
Tessa Gratton
Danielle Mages Amato
Sharon Huss Roat
Jilly Gagnon
Linsey Miller
Sierra Abrams
Christina June
Mechthild Gläser
Kristen Orlando
Gwen Cole
Tristina Wright
Whitney Gardner
Jenn Bennett
Lesley Livingston
Lexa Hillyer
Rachel E. Carter
Vic James
Lana Popović
Robin Roe
Sandhya Menon
Rin Chupeco
Karen M. McManus
Rosalyn Eves
Elly Blake
Tricia Levenseller
Amerie
Meagan Spooner
S. Jae-Jones
Jen  Klein
Fatih Turgut
Shelby Mahurin
Aysha Akhtar
Rachel  Harris
Kat Falls
Amy Mason Doan
R. Scott Bakker
Lev Grossman
Catherynne M. Valente
Susan Fletcher
Hans Christian Andersen
Cameron Dokey
Donna Jo Napoli
Elizabeth Marie Pope
Debbi

Katie McGarry
Cora Carmack
Nicole  Williams
Abbi Glines
Kristen Proby
Courtney Cole
Kendall Ryan
Tijan
Jennifer Iacopelli
Christina Lauren
Veronica Roth
Nyrae Dawn
J. Sterling
J. Lynn
E.L. James
Sylvia Day
Rebecca Donovan
James Daugherty
Dhan Gopal Mukerji
Erin Entrada Kelly
Joseph Krumgold
S.C. Stephens
Lynne Rae Perkins
Jamie McGuire
Elizabeth Yates
Nancy Willard
Ruth Sawyer
Elizabeth Enright
Walter D. Edmonds
Elizabeth Foreman Lewis
Emily Cheney Neville
Maia Wojciechowska
Kate Seredy
Scott Westerfeld
Cornelia Meigs
Dianne K. Salerni
George Orwell
Elizabeth Coatsworth
Ray Bradbury
Virginia Sorensen
Kelly Barnhill
Will James
Eleanor Estes
Elizabeth Gray Vining
Eric P. Kelly
Kate DiCamillo
Paula Fox
Frances Hodgson Burnett
Harold Keith
Ann Brashares
Betsy Byars
Cynthia Rylant
Robert Lawson
Margaret Peterson Haddix
Sara Shepard
Elizabeth Borton de Treviño
Jeanne Birdsall
Philip Pullman
Jack Gantos
Edward Bloor
Kirstin Pulioff
Jeff Kinney
Laura Amy Schlitz
Rachel Field
Lois Lenski
Laura 

Sam Kean
Daniel Kahneman
Oliver Sacks
Douglas R. Hofstadter
Charles Darwin
Jeff Lindsay
Liliana Hart
Angie Sage
Darren Shan
Lynne Ewing
David Eddings
Christopher Pike
Rachel Hawthorne
Brandon Mull
Jonathan Stroud
Carolyn Keene
Shanna Swendson
Yasmine Galenorn
James Howe
Mari Hamill
Barb Hendee
R.J. Craddock
Lenora Henson
Nora Roberts
Charlotte Blackwell
John Ajvide Lindqvist
Killion Slade
Annette Curtis Klause
D.M. Wolfenden
L.H. Cosway
Priscilla Glenn
Sawyer Bennett
Corinne Michaels
Calia Read
L.B. Simmons
Aly Martinez
Rebecca Yarros
Whitney G.
G.J. Walker-Smith
Tali Alexander
R.S. Grey
Beth Ehemann
Melanie Harlow
Kristen Callihan
Tillie Cole
Krista Ritchie
A.L. Jackson
Alessandra Torre
Renee Carlino
Brittainy C. Cherry
Jacquelyn Frank
Sasha Summers
Adrian Phoenix
Chris       Smith
Danielle DeVor
Lisa Desrochers
Dante Alighieri
Amber Kizer
Danielle Trussoni
Elizabeth Chandler
M.  Robinson
Nancy Haviland
Lisa Cardiff
S.W. Frank
Francette Phal
Bethany-Kris
Sienna Mynx
Soraya Naomi
Jessi

Timothy Snyder
Doris Kearns Goodwin
Ian Kershaw
Simon Schama
Mark Kurlansky
Michel Foucault
Ron Chernow
Greg King
Antony Beevor
Eugene B. Sledge
Richard J. Evans
Tom Holland
Stephen E. Ambrose
Nathaniel Philbrick
Vincent Bugliosi
Hampton Sides
John Farman
S.C. Gwynne
Edmund Morris
Suetonius
James W. Loewen
Tim Weiner
Antonia Fraser
Timothy Egan
Jack Weatherford
David Fromkin
Iris Chang
Christopher   Clark
Thomas J. DiLorenzo
Margaret MacMillan
Thucydides
Charles C. Mann
Eamon Duffy
John Newsinger
Herodotus
Erik Larson
James M. McPherson
Dee Brown
Lynn Austin
Lori Wick
Ted Dekker
Cathy Marie Hake
Karen Kingsbury
Julie Klassen
Charles Martin
Randy Alcorn
Elizabeth Payson Prentiss
Cindy Woodsmall
Donita K. Paul
Terri Blackstock
Bodie Thoene
Julie Lessman
Beverly Lewis
Eric Wilson
Liz Curtis Higgs
Jan Karon
J.J.   Johnson
Dee Henderson
Deeanne Gist
Shelly Goodman Wright
Catherine Marshall
Adriana Trigiani
Eudora Welty
Alexandra Ripley
Sheri Reynolds
Joshilyn Jackson
Erskine Caldwell
Kaye G

...

Todd Strasser
Swati Avasthi
Cindy C. Bennett
Gail Giles
Edward Bloor
Carol Plum-Ucci
V.C. Andrews
C.L. Heckman
Catherine Atkins
Barry Lyga
Laura Wiess
Elizabeth Scott
Dave Pelzer
Christine Rimmer
Mike Mullin
Nikki McCoy
Amelia Hutchins
Ralph Waldo Emerson
Sarah McCarty
Kathleen Norris
Sean Smith
Glenn Beck
Dick Morris
Algis Budrys
James Frey
Ivana Trump
Russell Brand
William A. Dembski
L. Ron Hubbard
Rick Warren
Sean Hannity
Karl Rove
Ayn Rand
Michael Savage
Oprah Winfrey
Ann Coulter
Anton Szandor LaVey
Phillip E. Johnson
Barack Obama
Al Franken
Sarah Palin
Al Gore
Miley Cyrus
Candace Bushnell
Amy Sohn
Hillary Rodham Clinton
Kaylene Johnson
Paris Hilton
Tim LaHaye
John Dominic Crossan
Dan C. Quayle
Adolf Hitler
Christopher Hitchens
The Silver Elves
Miguel Ruiz
Nicole Snooki Polizzi
Ronald Hutton
Macel Ely II
Madonna
Howard Stern
Daniel C. Dennett
Karen Armstrong
Isaac Asimov
Samuel Beckett
Carl Jung
Thomas  Moore
Joe Hilley
Alexandra Monir
Deepak Chopra
Bart D. Ehrman
Rush Limbaugh
Kei

Mark Kurlansky
Erlend Loe
Vincent Bugliosi
Suzanne Woods Fisher
Peter Shaffer
M.T. Anderson
Joel C. Rosenberg
Kenneth Grahame
Sandra Orchard
Patricia  Bradley
Mary  Ellis
Krista McGee
Elizabeth Ludwig
Jane Kirkpatrick
Cyn Balog
Alys Arden
Julianna Deering
J.B. McGee
Lisa J. Lickel
Kathleen Y'Barbo
Susie Finkbeiner
Cat Phoenix
Cynthia Ruchti
Leslie Gould
Heather Brewer
Krista Ritchie
Courtney Cole
Beth Wiseman
CLAMP
Eiichiro Oda
Kate Breslin
Kyousuke Motomi
Tsugumi Ohba
Peach-Pit
Atsushi Ohkubo
Naoko Takeuchi
Kanoko Sakurakouji (桜小路かのこ)
Meca Tanaka
Hisamu Oto
Mika Yamamori
Jun Mochizuki
Julietta Suzuki
Masashi Kishimoto
Yuu Watase
Rumiko Takahashi
Yoshihiro Togashi
fallenbabybubu
Tomoko Hayakawa
Natsumi Andō
Kazune Kawahara
Yōko Kamio
Hiro Fujiwara
HaveYouSeenThisGirL
Beth Reekles
Kagami Yoshimizu
Supraja I.R.
Mina V. Esguerra
Aril Daine
Alesana Marie
Jessica Concha
Jade Margarette Pitogo
Kimberly Joy Villanueva
Alyloony
Chad Harbach
Joshilyn Jackson
Téa Obreht
Rosamund Lupton
Jenna Blu

Ovid
Billy Collins
Natsuki Takaya
Gavin MacLeod
Wisława Szymborska
Bill Clinton
John Elder Robison
Leonard Cohen
Solomon Northup
Anna Akhmatova
Sidney Poitier
Michael Patrick MacDonald
Ali Marsman
Langston Hughes
Nelson Mandela
W.B. Yeats
D.L. McDermott
Benjamin Franklin
Russell Baker
Ken Dickson
Robert Frost
John Keats
E.E. Cummings
Russel Lazega
Friedrich Schiller
Malcolm X
Peter   John
Sandy Williams
Ezra Pound
Elizabeth   Hunter
Charles Baudelaire
Brett J. Talley
Nick Cave
Friedrich Hölderlin
Allen Ginsberg
Phillip W. Simpson
Taryn Browning
R.A. Gates
Danelle O'Donnell
Emily Dickinson
Eddie Huang
Johan Harstad
Carolyn Crane
Suzanne McLeod
David Litwack
Maria E. Schneider
Robyn Roze
Caroline Hanson
Sierra Dean
Helen Boswell
Aurelia B. Rowl
Wendy S. Russo
Jennifer Szewczuk
Kaitlyn Oruska
Annie Brewer
Anna Fienberg
Michael Cart
J. Elizabeth Hill
Melody Carlson
Antonia Michaelis
Su Williams
Lauren Strasnick
Ann Head
Tanya Lee Stone
Maer Wilson
Kerry Cohen
S.R. Mallery
Maria Rachel Hool

Kate Paddington
Belinda Burke
Lilah Suzanne
James Faro
Teodora Kostova
Anne O'Gleadra
Calia Read
Kathleen Kerridge
Thorny Sterling
Danielle Stinson
Rafe Haze
David Cordingly
Will Freshwater
E.S. Skipper
Diana Preston
Tessa Teevan
Margaret McHeyzer
Hergé
Howard Pyle
A.J. DeWall
Samantha Hastings
S.E. Harmon
Erin  Stewart
Ella  Stone
Dugald A. Steer
Zarah Detand
Rachel Hollis
Brittney Morris
Sarah Ann Juckes
Kip Wilson
Emma Grayson
A.C. Crispin
Ben Philippe
Kim Smejkal
Karen Perkins
Shana Silver
Stephan Talty
Michelle Ruiz Keil
Ryan Michele
Kara McDowell
Rob Kidd
Catherine Tinker
Lillian  Clark
Rena Barron
Johann David Wyss
Bridget Tyler
Dhonielle Clayton
Stacey Lewis
Sarah  Carlson
Jodie Lynn Zdrok
Lillie Vale
Katy Loutzenhiser
Lisa Suzanne
Margaret  Owen
Opal Carew
Addie Thorley
Herbie Brennan
Bárbara G. Rivero
Erin A. Craig
Laura Pohl
Margaret  Rogerson
Hanna Alkaf
Tiana Smith
Kalyn Josephson
Tehlor Kay Mejia
Isabel Sterling
Jayne Rylon
Catherine Cookson
Justin A. Reynolds
Neil deGras

Ian McGuire
Antonia Hayes
Jeffrey Brown
Viviane Schwarz
A.A. McDonald
Michelle Adelman
Antonia Barber
Emily Cheney Neville
William Kent Krueger
Elizabeth Coatsworth
Reed Farrel Coleman
The New Yorker
Susan Minot
Kristopher Jansma
Rachel Cusk
Miranda Beverly-Whittemore
Xiaolu Guo
Tara  Sullivan
Fuminori Nakamura
Robert  Allison
Gerard Woodward
B. Kliban
James          Anderson
Bruce Holsinger
Harriet Lane
Ron Rash
Alex Preston
Sadie Jones
Jennifer Erin Valent
Christine  Reilly
Paul Bailey
Caroline Paul
Carolyn Jourdan
Tobias Hill
Fred Chappell
Rachel Cantor
Seré Prince Halverson
Monica Wood
Joshua Ferris
Helen Klein Ross
Kanata Konami
Naomi Wood
S.F. Said
Teju Cole
Hanif Kureishi
Cynan Jones
W.B. Belcher
Erma Bombeck
Taylor  Brown
Michael Lee West
Akhil   Sharma
Jason Gurley
Adriana Trigiani
Joseph O'Neill
Nic Schuck
Linda Grant
Paul Kingsnorth
Spike Milligan
John Ehle
Sara Walsh
Amélie Nothomb
Laurie Notaro
Thierry Jonquet
Will Cuppy
Ann Hite
Simon Tofield
Margaret Drabble
Audrey Magee

In [12]:
goodread_books_df = cleaned_goodreads_df[["book_title","author_name","genre_1", "genre_2", "book_average_rating", 
                                          "num_ratings", "num_reviews", "pages", "publish_date"]].copy()
goodread_books_df.head()

,book_title,author_name,genre_1,genre_2,book_average_rating,num_ratings,num_reviews,pages,publish_date
0,Murder on St. Mark's Place,Victoria Thompson,Mystery,Historical,4.02,5260,375,277,2000
1,The Girl with the Dragon Tattoo,Stieg Larsson,Fiction,Mystery,4.13,2229163,65227,465,2005
2,Tailored for Trouble,Mimi Jean Pamfiloff,Romance,Contemporary,3.99,2151,391,354,2016
3,The Obscene Bird of Night,José Donoso,Fiction,Magical Realism,4.14,1844,173,438,1970
4,Sorcery & Cecelia: or The Enchanted Choc...,Patricia C. Wrede,Fantasy,Young Adult,4.01,17051,1890,326,1988


In [13]:
# Database connection string
rds_connection_string = f"{user_id}:{password}@127.0.0.1/good_books_db"
engine = create_engine(f'mysql://{rds_connection_string}?charset=utf8&use_unicode=1')

In [14]:
# Display tables
engine.table_names()

['author', 'book_gdr', 'book_nyt']

In [15]:
## Preparing good read database information for authors data
goodread_authors_df = authors_df[["author_name", "author_gender", "author_average_rating", "author_rating_count"]]
goodread_authors_df["type"] = "GDR"
goodread_authors_df.rename(columns={"author_name": "name" , 
                                                "author_gender" : "gender", 
                                                "author_average_rating" : "rating", 
                                                "author_rating_count": "count"}, inplace=True)
goodread_authors_df.head()

,name,gender,rating,count,type
0,Victoria Thompson,female,4.01,74399,GDR
1,Stieg Larsson,male,4.15,3726435,GDR
2,Mimi Jean Pamfiloff,female,4.00,76496,GDR
3,José Donoso,male,3.88,5522,GDR
4,Patricia C. Wrede,female,4.10,291013,GDR


In [16]:
###### author #####
# Insert goodread_authors to authors table
num_authors = len(goodread_authors_df)

#Iterate one row at a time
for i in range(num_authors):
    try:
        #Try inserting the row
        goodread_authors_df.iloc[i:i+1].to_sql(name="author",con = engine,if_exists = 'append',index=False)
    except :
        #Ignore the rows if exception
#         print("Skipped the data - " + goodread_authors_df.iloc[i:i+1])
        pass

In [17]:
## Preparing nyt database information for authors data
nyt_authors_df = nyt_transformed[["Author"]]
nyt_authors_df.rename(columns={"Author": "name"},inplace=True)
nyt_authors_df["gender"] = None
nyt_authors_df["rating"] = None
nyt_authors_df["count"] = None
nyt_authors_df["type"] = "NYT"
nyt_authors_df.head()

/Users/vinay/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/vinay/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/vinay/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,name,gender,rating,count,type
0,Dean R Koontz,None,None,None,NYT
1,Stephenie Meyer,None,None,None,NYT
2,Emily Giffin,None,None,None,NYT
3,Patricia Cornwell,None,None,None,NYT
4,Chuck Palahniuk,None,None,None,NYT


In [18]:
###### author #####
# Insert nyt_authors to authors table
num_authors = len(nyt_authors_df)

#Iterate one row at a time
for i in range(num_authors):
    try:
        #Try inserting the row
        nyt_authors_df.iloc[i:i+1].to_sql(name="author",con = engine,if_exists = 'append',index=False)
    except :
        #Ignore the rows if exception
#         print("Skipped the data - " + nyt_authors_df.iloc[i:i+1])
        pass

In [36]:
goodread_books_df.head()

,book_title,author_name,genre_1,genre_2,book_average_rating,num_ratings,num_reviews,pages,publish_date,New title
0,Murder on St. Mark's Place,Victoria Thompson,Mystery,Historical,4.02,5260,375,277,2000,MURDER ON ST. MARK'S PLACE
1,The Girl with the Dragon Tattoo,Stieg Larsson,Fiction,Mystery,4.13,2229163,65227,465,2005,THE GIRL WITH THE DRAGON TATTOO
2,Tailored for Trouble,Mimi Jean Pamfiloff,Romance,Contemporary,3.99,2151,391,354,2016,TAILORED FOR TROUBLE
3,The Obscene Bird of Night,José Donoso,Fiction,Magical Realism,4.14,1844,173,438,1970,THE OBSCENE BIRD OF NIGHT
4,Sorcery & Cecelia: or The Enchanted Choc...,Patricia C. Wrede,Fantasy,Young Adult,4.01,17051,1890,326,1988,SORCERY & CECELIA: OR THE ENCHANTED CHOC...


In [19]:
###### book_gdr #####
num_rows = len(goodread_books_df)

#Iterate one row at a time
for i in range(num_rows):
    try:
        #Try inserting the row
        goodread_books_df.iloc[i:i+1].to_sql(name="book_gdr",con = engine,if_exists = 'append',index=False)
    except :
        #Ignore duplicates
#         print("Skipped the data - " + goodread_books_df.iloc[i:i+1])
        pass

In [21]:
## Preparing nyt database information for authors data
nyt_books_df = nyt_transformed[["Title", "Author","Published year"]]
nyt_books_df.rename(columns={"Title": "book_title", 
                             "Author":"author_name",
                             "Published year": "publish_year"},inplace=True)
nyt_books_df.head()

/Users/vinay/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,book_title,author_name,publish_year
0,ODD HOURS,Dean R Koontz,2008
1,THE HOST,Stephenie Meyer,2008
2,LOVE THE ONE YOU'RE WITH,Emily Giffin,2008
3,THE FRONT,Patricia Cornwell,2008
4,SNUFF,Chuck Palahniuk,2008


In [22]:
###### book_nyt #####
num_rows = len(nyt_books_df)

#Iterate one row at a time
for i in range(num_rows):
    try:
        #Try inserting the row
        nyt_books_df.iloc[i:i+1].to_sql(name="book_nyt",con = engine,if_exists = 'append',index=False)
    except :
        #Ignore duplicates
#         print("Skipped the data - " + goodread_books_df.iloc[i:i+1])
        pass

In [29]:
pd.read_sql_query('select * from author', con=engine).head()

,id,name,gender,rating,count,type
0,1,Victoria Thompson,female,4.01,74399.0,GDR
1,2,Stieg Larsson,male,4.15,3726435.0,GDR
2,3,Mimi Jean Pamfiloff,female,4.00,76496.0,GDR
3,4,José Donoso,male,3.88,5522.0,GDR
4,5,Patricia C. Wrede,female,4.10,291013.0,GDR


In [30]:
pd.read_sql_query('select * from book_gdr', con=engine).head()

,id,book_title,author_name,genre_1,genre_2,book_average_rating,num_ratings,num_reviews,pages,publish_date
0,1,Murder on St. Mark's Place,Victoria Thompson,Mystery,Historical,4.02,5260,375,277,2000.0
1,2,The Girl with the Dragon Tattoo,Stieg Larsson,Fiction,Mystery,4.13,2229163,65227,465,2005.0
2,3,Tailored for Trouble,Mimi Jean Pamfiloff,Romance,Contemporary,3.99,2151,391,354,2016.0
3,4,The Obscene Bird of Night,José Donoso,Fiction,Magical Realism,4.14,1844,173,438,1970.0
4,5,Sorcery & Cecelia: or The Enchanted Choc...,Patricia C. Wrede,Fantasy,Young Adult,4.01,17051,1890,326,1988.0


In [31]:
pd.read_sql_query('select * from book_nyt', con=engine).head()

,id,book_title,author_name,publish_year
0,1,ODD HOURS,Dean R Koontz,2008
1,2,THE HOST,Stephenie Meyer,2008
2,3,LOVE THE ONE YOU'RE WITH,Emily Giffin,2008
3,4,THE FRONT,Patricia Cornwell,2008
4,5,SNUFF,Chuck Palahniuk,2008
